# Markup utilities  
> Functions for marking up label images using a trained model

In [1]:
#hide
# default_exp markup_utils
from nbdev.showdoc import *
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Setup environment



### Imports

In [ ]:
# hide
# hidden imports
from  cellsegment.markup_utils import *   #hide this otherwise recurseive 

In [3]:
#export

from cellsegment.core import *
from cellsegment.inference_utils import *
from cellsegment.json_utils import *
from cellsegment.dataprep_utils import *
from cellsegment.set_directories import *
import cv2
from skimage.measure import label, regionprops
import pandas
from fastai.vision import *

## Define directories


In [4]:
img_path = Path('nbs_test_data')
mrk_path = Path('nbs_test_data/markup')  
mrk_path.mkdir(parents=True, exist_ok=True)
fnames = sorted(get_image_files(img_path))
fn = fnames[1]
fn

PosixPath('nbs_test_data/236568 - 10.jpg')

In [5]:
fnames

[PosixPath('nbs_test_data/235443 - 1.jpg'),
 PosixPath('nbs_test_data/236568 - 10.jpg')]

## utilities

In [6]:
# export
def padImage_t(img, pad=100):
    if pad and pad > 0:
        return F.pad(input=img.px, pad=(pad, pad, pad, pad), mode='constant', value=0)
    else:
        return img

def cut_tiles_t(img, TM=4, TN=4, pad=100):
    M, N = (img.shape[1]-pad*2)//TM, (img.shape[2]-pad*2)//TN
    OM, ON = pad + M//2, pad + N//2
    return [Image(img[:,x-OM:x+OM,y-ON:y+ON]) for x in range(pad+M//2,img.shape[1],M) for y in range(pad+N//2,img.shape[2],N)]

def lay_tiles_t(tiles, TM=4, TN=4, pad=100):
    (_,M,N) = tiles[0].size()
    OM, ON = pad + M//2, pad + N//2
    for n, tile in enumerate(tiles):
        tiles[n] = tile[:,pad:-(pad+1),pad:-(pad+1)]

    hstack = [torch.cat(tiles[y:y+TN],dim=2) for y in range(0,TN*TM,TN)]
    return torch.cat(hstack,dim=1)

def run_prediction(learn, fn, filesavedir=None, tile=False, pad=100):
    """
    run predictions and return dictionary of raw predictions
    - learn: learner to use
    - fn: filename
    - filesavedir: if set will save to this directory
    - tile: use tiling or not
    - pad: padding factor for tiling
    - return rediction tensor
    """
    img = open_image(fn)
    if tile:
        pad = 100
        TM, TN =2, 2
        img = padImage_t(img, pad=pad)
        tiles = cut_tiles_t(img, TM=TM, TN=TN, pad=pad)
        outputs = [learn.predict(im)[2] for im in tiles]
        raw_pred = lay_tiles_t(outputs, TM=TM, TN=TN, pad=pad)
    else:
        if img.shape[1]%2 == 1:   # make odd dimension make even for unet
           img.px = img.px[:,1:,:]
        if img.shape[2]%2 == 1:   # make odd dimension make even for unet
           img.px = img.px[:,:,1:]
        _,_,raw_pred = learn.predict(img)

    preds = {c[0]:raw_pred[i,:,:] for i,c in enumerate(CLASS_LABELS.items())}
    if filesavedir:
        for cls,raw_pred in preds.items():
            save_png_p(to_np(raw_pred), f'{filesavedir}/{fn.stem}-{cls}.png')

    return preds

In [7]:
print("Class Labels")
CLASS_LABELS

Class Labels


{'Background': {'Code': '0', 'Fill': (0, 0, 0, 127)},
 'Liver Fluke': {'Code': '11', 'Fill': (255, 0, 0, 127)},
 'Rumen Fluke': {'Code': '40', 'Fill': (0, 255, 0, 127)},
 'Other': {'Code': '255', 'Fill': (0, 0, 255, 127)}}

In [8]:
# export
def find_prediction_blobs( cls, img,  min_conf=0.1, min_area=100):
    'find prediction blobs, filter by confidence and area'
    SCALE = 100
    min_conf *= SCALE

    img = (img * (SCALE/img.max())).astype(np.int)
    img[img<min_conf] = 0
    predictions = []
    label_image = label(img > min_conf)

    #         img = np.array(img) # helped with a cv error?
    regions = regionprops(label_image, img, cache=True)
    for region in regions:
        if region.area > min_area:
            cx = int(region.centroid[1])
            cy = int(region.centroid[0])
            predictions.append({"label": cls, "point": [cx,cy], "probability": region.max_intensity.round(2)})

    return predictions, img, regions

### Test find_prediction_blobs( cls, img,  min_conf=0.1, min_area=100)

In [9]:
 find_prediction_blobs( cls, img,  min_conf=0.1, min_area=100)

In [ ]:
# export
def annotate_json(jdata, classes, annotations):
    'Annotate the json file with predictions'
    def add_anno(data, item):
        r = 2  # this is just used as a center postion
        cx,cy = item['point']
        pnt_list = [[cx-r,cy-r], [cx+r,cy+r]]

        probability = str(item['probability']) if 'probability' in item else str(0)

        try:
            it = CLASS_LABELS[item['label']]
            line_color = it['Fill']
            data['shapes'].append({
                    "label": item['label'],"line_color": line_color, "fill_color": None,
                    "points": pnt_list, "shape_type": "rectangle", 'probability': probability
                })
        except:
            print("Error in parsing CLASS_LABELS")

    def add_annotations(data, annotations):
        for item in annotations:
            add_anno(data, item)

    def del_circle_annotations(data):
        to_del = [s for s,sh in enumerate(data['shapes']) if sh['shape_type']=='circle']
        if len(to_del) > 0:
            print(f'Deleting {len(to_del)} circle annotations')
        for i in sorted(to_del, reverse=True):
            del data['shapes'][i]

    # data = json.load(open(fn))
    # del_circle_annotations(data)
    add_annotations(jdata, annotations)
    return jdata

In [10]:
# export
def calc_probs(raw_pred:tensor)->list:
    """
    Calculate the probabilities from a raw prediction  `raw_pred`
    by finding the average value of all the pixels within a region
    This assumes that there is only one cell in the image
    - `raw_pred`: the raw predictions from the learner
    - `return`:  list of probabilities
    """
    def __get_bool_mask(o):
        min_conf = 0.5
        arr = o[0, :, :].numpy()
        bgnd = get_region_props(o, 0, min_conf)
        if len(bgnd) > 0:
            mask = np.zeros((arr.shape), dtype=bool)
            AAcoords = bgnd[0]["coords"]
            mask[tuple(AAcoords.T.tolist())] = True
        else:
            mask = None
        return mask

    mask = __get_bool_mask(raw_pred)
    if mask is not None:
        p = [None] * raw_pred.shape[0]
        for i in range(raw_pred.shape[0]):
            p[i] = raw_pred[i, :, :].numpy()[mask].mean().round(2)
    else:
        p = []
    return p

In [11]:
# export
def draw_labels_cv(img, json, radius=20):
    font = cv2.FONT_HERSHEY_SIMPLEX
    for s, sh in enumerate(json['shapes']):

        fill = sh["line_color"]
        draw = sh['shape_type']
        probability = str(sh['probability']) if 'probability' in sh else ''

        xy = np.asarray(sh["points"])
        ave = np.mean(xy,axis=0)

        cx = int(ave[0])
        cy = int(ave[1])

        if draw == 'circle':
            cv2.circle(img, (cx, cy), radius, fill, 2)
            cv2.circle(img, (cx, cy), radius, fill, 2)
            cv2.putText(img,probability,(int(cx-radius), cy-radius), font, 1, fill, 2, cv2.LINE_AA)

        elif draw == 'rectangle':
            cv2.rectangle(img, (cx - radius, cy - radius), (cx + radius, cy + radius), fill, 2)
            cv2.putText(img,probability,(int(cx-radius), cy-radius), font, 1, fill, 2, cv2.LINE_AA)

    return img

In [12]:
# export

def markup_one_image(fn:str, preds:dict):
    src_img = np.asarray(PIL.Image.open(fn))
    # find blobs in png            mrk_img, jdata = self.markup_image(img)
    # preds.items()[1]
    pred_list = []
    for cls, img in preds.items():
        if cls != 'Background':
            # print(f'Processing {cls}')
            prd, _, _ = find_prediction_blobs(cls, to_np(img))
            pred_list += prd

    jdata = createjson(fn=fn.name, offset=0)
    jdata= annotate_json(jdata, CLASS_LABELS, pred_list)
    # print(pred_list)
    # print(prettyjson(jdata))
    # with open(f'{mrk_path}/{fn.stem}.json', 'w') as outfile:
    #     json.dump(jdata, outfile, ensure_ascii=False, indent=4)
    # mark up jpg
    mrk_img = np.asarray(PIL.Image.open(fn))
    mrk_img = draw_labels_cv(mrk_img, jdata, radius=50)
    return mrk_img, pred_list, jdata
    # # PIL.Image.fromarray(mrk_img.astype(np.uint8)).save(f'{mrk_path}/{fn.stem}.jpg', quality=90)